<h1>Comparison of Transfer Learning Models - Occipital View </h1>

<hr>

<p>Continuing with the model selection process, we proceed to train the three models demonstrating the highest accuracy from the previous step over a span of 10 epochs. It's important to note that, at present, we are not employing any form of data augmentations.</p>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import inspect
import glob
import os
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
import pandas as pd
import inspect
from tqdm import tqdm
from keras.preprocessing import image
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from google.colab import drive
from sklearn.model_selection import StratifiedKFold

In [ ]:
from keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
batch_size = 32
data_path = "/content/drive/MyDrive/sve_O - bez augumentacija/Train"
n_folds =5
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
all_files = glob.glob(os.path.join(data_path, '*/*'))
labels = [os.path.basename(os.path.dirname(fp)) for fp in all_files]

In [ ]:
# List all available models
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [ ]:
len(model_dictionary)

71

In [ ]:
new_model_dict = {key: model_dictionary[key] for key in model_dictionary  if key in ['MobileNet', 'ResNet101V2','ResNet50V2']}

In [ ]:
len(new_model_dict)

3

In [ ]:
def preprocess_data_for_fold(train_files, val_files, img_size):
    #datagen = image.ImageDataGenerator(rescale=1./255., rotation_range=15, brightness_range=[0.2, 1.0])
    datagen = image.ImageDataGenerator(rescale=1./255.)

    # Extracting the labels from the directory structure
    train_labels = [os.path.basename(os.path.dirname(fp)) for fp in train_files]
    val_labels = [os.path.basename(os.path.dirname(fp)) for fp in val_files]

    train_gen = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': train_files, 'class': train_labels}),
        directory=None,
        x_col="filename",
        y_col="class",
        class_mode="binary",
        target_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )

    val_gen = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_files, 'class': val_labels}),
        directory=None,
        x_col="filename",
        y_col="class",
        class_mode="binary",
        target_size=img_size,
        batch_size=batch_size,
        shuffle=False
    )

    return train_gen, val_gen


In [ ]:
def test_preprocces(img_size):
  test_datagen = image.ImageDataGenerator(rescale=1./255.)
  test_generator = test_datagen.flow_from_directory("/content/drive/MyDrive/sve_O - bez augumentacija/Test",
                                        target_size=img_size,
                                        class_mode='binary',
                                        batch_size=batch_size,
                                        shuffle=False)
  return test_generator

In [ ]:
def build_model(model_1,input_shape):
    pre_trained_model = model_1(include_top=False, pooling='avg', input_shape=input_shape)
    pre_trained_model.trainable = False

    model = tf.keras.models.Sequential()
    model.add(pre_trained_model)
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Loop over each model available in Keras
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': [], 'test_accuracy': []}
for model_name, model_1 in tqdm(list(new_model_dict.items())):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)

    fold_no = 1
    all_acc = []
    all_loss = []

    for train_indices, val_indices in kfold.split(all_files,labels):
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

    # Preprocess data and build model (assuming these functions are defined)
        train_files = [all_files[i] for i in train_indices]
        val_files = [all_files[i] for i in val_indices]
        if 'NASNetLarge' in model_name:
          img_size =(331,331)
          input_shape =(331,331,3)
          train_gen, val_gen = preprocess_data_for_fold(train_files, val_files, img_size)
        elif 'InceptionResNetV2' or 'InceptionV3' or 'Xception' in model_name:
          img_size=(299,299)
          input_shape=(299,299,3)
          train_gen, val_gen = preprocess_data_for_fold(train_files, val_files, img_size)
        else:
          img_size=(224,224)
          input_shape=(224,224,3)
          train_gen, val_gen = preprocess_data_for_fold(train_files, val_files, img_size)


        model = build_model(model_1,input_shape)

    # Fit data to model
        history = model.fit(train_gen, epochs=10, validation_data=val_gen)

    # Evaluate the model on validation set
        scores = model.evaluate(val_gen, verbose=0)
        print(f'Score for fold {fold_no}: Loss of {scores[0]}; Accuracy of {round(scores[1]*100, 2)}%')
        all_acc.append(scores[1] * 100)
        all_loss.append(scores[0])

    # Increase fold number
        fold_no += 1

# Provide average scores
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(len(all_acc)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {all_loss[i]} - Accuracy: {all_acc[i]}%')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(all_acc)} (+- {np.std(all_acc)})')
    print(f'> Loss: {np.mean(all_loss)}')
    print('------------------------------------------------------------------------')
    if 'NASNetLarge' in model_name:
        test_processed = test_preprocces(img_size)
    elif 'InceptionResNetV2' or 'InceptionV3' or 'Xception' in model_name:
        test_processed = test_preprocces(img_size)
    else:
        test_processed = test_preprocces(img_size)

    test_acc = model.evaluate(test_processed)
    predictions = model.predict(test_processed)
    predicted_classes = np.where(predictions < 0.5, 0, 1)
    true_classes = test_processed.classes
    class_labels = list(test_processed.class_indices.keys())

    report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
    print(report)

    # Calculate all relevant metrics
    model_benchmarks['model_name'].append(model_name)
    model_benchmarks['num_model_params'].append(model.count_params())
    model_benchmarks['validation_accuracy'].append(np.mean(all_acc))
    model_benchmarks['test_accuracy'].append(test_acc[-1])


  0%|          | 0/3 [00:00<?, ?it/s]

------------------------------------------------------------------------
Training for fold 1 ...
Found 125 validated image filenames belonging to 2 classes.
Found 32 validated image filenames belonging to 2 classes.


17225924/17225924 [==============================] - 0s 0us/step
Epoch 1/10
4/4 [==============================] - 29s 7s/step - loss: 0.7495 - accuracy: 0.4240 - val_loss: 0.6979 - val_accuracy: 0.4375
Epoch 2/10
4/4 [==============================] - 2s 519ms/step - loss: 0.7125 - accuracy: 0.4960 - val_loss: 0.6809 - val_accuracy: 0.6875
Epoch 3/10
4/4 [==============================] - 2s 537ms/step - loss: 0.6962 - accuracy: 0.5120 - val_loss: 0.6703 - val_accuracy: 0.6562
Epoch 4/10
4/4 [==============================] - 2s 413ms/step - loss: 0.6757 - accuracy: 0.5920 - val_loss: 0.6607 - val_accuracy: 0.6250
Epoch 5/10
4/4 [==============================] - 2s 416ms/step - loss: 0.6593 - accuracy: 0.6240 - val_loss: 0.6571 - val_accuracy: 0.6250
Epoch 6/10
4/4 [==============================] - 2s 416ms/step - loss: 0.6394 - accuracy: 0.6960 - val_loss: 0.6433 - val_accuracy: 0.5000
Epoch 7/10
4/4 [==============================] - 2s 413ms/step - loss: 0.6233 - accuracy: 0.7280

Epoch 1/10
4/4 [==============================] - 4s 603ms/step - loss: 0.6991 - accuracy: 0.5760 - val_loss: 0.7311 - val_accuracy: 0.5938
Epoch 2/10
4/4 [==============================] - 2s 416ms/step - loss: 0.6738 - accuracy: 0.6160 - val_loss: 0.7178 - val_accuracy: 0.4375
Epoch 3/10
4/4 [==============================] - 2s 503ms/step - loss: 0.6491 - accuracy: 0.6720 - val_loss: 0.7126 - val_accuracy: 0.4375
Epoch 4/10
4/4 [==============================] - 2s 585ms/step - loss: 0.6321 - accuracy: 0.7200 - val_loss: 0.7064 - val_accuracy: 0.4062
Epoch 5/10
4/4 [==============================] - 2s 411ms/step - loss: 0.6150 - accuracy: 0.7440 - val_loss: 0.7022 - val_accuracy: 0.4375
Epoch 6/10
4/4 [==============================] - 2s 407ms/step - loss: 0.6010 - accuracy: 0.7680 - val_loss: 0.6975 - val_accuracy: 0.4375
Epoch 7/10
4/4 [==============================] - 2s 418ms/step - loss: 0.5872 - accuracy: 0.7840 - val_loss: 0.6939 - val_accuracy: 0.5312
Epoch 8/10
4/4 [====

Epoch 1/10
4/4 [==============================] - 6s 1s/step - loss: 0.8025 - accuracy: 0.5000 - val_loss: 0.7132 - val_accuracy: 0.5161
Epoch 2/10
4/4 [==============================] - 2s 399ms/step - loss: 0.7172 - accuracy: 0.4921 - val_loss: 0.7452 - val_accuracy: 0.5161
Epoch 3/10
4/4 [==============================] - 2s 416ms/step - loss: 0.7247 - accuracy: 0.5079 - val_loss: 0.7069 - val_accuracy: 0.5161
Epoch 4/10
4/4 [==============================] - 2s 514ms/step - loss: 0.6926 - accuracy: 0.5397 - val_loss: 0.6879 - val_accuracy: 0.4839
Epoch 5/10
4/4 [==============================] - 2s 532ms/step - loss: 0.6752 - accuracy: 0.5714 - val_loss: 0.6804 - val_accuracy: 0.5161
Epoch 6/10
4/4 [==============================] - 2s 405ms/step - loss: 0.6655 - accuracy: 0.5794 - val_loss: 0.6661 - val_accuracy: 0.7419
Epoch 7/10
4/4 [==============================] - 2s 415ms/step - loss: 0.6448 - accuracy: 0.6746 - val_loss: 0.6596 - val_accuracy: 0.6452
Epoch 8/10
4/4 [=======

...

              precision    recall  f1-score   support

           F       0.66      0.82      0.73        33
           M       0.77      0.59      0.67        34

    accuracy                           0.70        67
   macro avg       0.71      0.70      0.70        67
weighted avg       0.71      0.70      0.70        67

------------------------------------------------------------------------
Training for fold 1 ...
Found 125 validated image filenames belonging to 2 classes.
Found 32 validated image filenames belonging to 2 classes.
171317808/171317808 [==============================] - 1s 0us/step
Epoch 1/10
4/4 [==============================] - 18s 2s/step - loss: 0.7535 - accuracy: 0.4640 - val_loss: 0.7385 - val_accuracy: 0.4062
Epoch 2/10
4/4 [==============================] - 2s 514ms/step - loss: 0.7280 - accuracy: 0.4720 - val_loss: 0.7133 - val_accuracy: 0.4688
Epoch 3/10
4/4 [==============================] - 2s 512ms/step - loss: 0.6791 - accuracy: 0.5920 - val_loss: 0.

 67%|██████▋   | 2/3 [06:36<03:19, 199.71s/it]

              precision    recall  f1-score   support

           F       0.78      0.55      0.64        33
           M       0.66      0.85      0.74        34

    accuracy                           0.70        67
   macro avg       0.72      0.70      0.69        67
weighted avg       0.72      0.70      0.69        67

------------------------------------------------------------------------
Training for fold 1 ...
Found 125 validated image filenames belonging to 2 classes.
Found 32 validated image filenames belonging to 2 classes.
94668760/94668760 [==============================] - 0s 0us/step
Epoch 1/10
4/4 [==============================] - 7s 830ms/step - loss: 0.7165 - accuracy: 0.4720 - val_loss: 0.7119 - val_accuracy: 0.5625
Epoch 2/10
4/4 [==============================] - 2s 417ms/step - loss: 0.6821 - accuracy: 0.6480 - val_loss: 0.6803 - val_accuracy: 0.5625
Epoch 3/10
4/4 [==============================] - 2s 434ms/step - loss: 0.6365 - accuracy: 0.6720 - val_loss: 0.

100%|██████████| 3/3 [09:06<00:00, 182.04s/it]

              precision    recall  f1-score   support

           F       0.69      0.82      0.75        33
           M       0.79      0.65      0.71        34

    accuracy                           0.73        67
   macro avg       0.74      0.73      0.73        67
weighted avg       0.74      0.73      0.73        67



In [ ]:
# Convert Results to DataFrame for easy viewing
benchmark_df = pd.DataFrame(model_benchmarks)
benchmark_df.sort_values('num_model_params', inplace=True) # sort in ascending order of num_model_params column
benchmark_df

,model_name,num_model_params,validation_accuracy,test_accuracy
0,MobileNet,3229889,63.124999,0.701493
2,ResNet50V2,23566849,75.221775,0.731343
1,ResNet101V2,42628609,80.282258,0.701493


from matplotlib import pyplot as plt
benchmark_df['num_model_params'].plot(kind='hist', bins=20, title='num_model_params')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
benchmark_df['validation_accuracy'].plot(kind='hist', bins=20, title='validation_accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
benchmark_df['test_accuracy'].plot(kind='hist', bins=20, title='test_accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
benchmark_df.groupby('model_name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
benchmark_df['validation_accuracy'].plot(kind='line', figsize=(8, 4), title='validation_accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
benchmark_df['test_accuracy'].plot(kind='line', figsize=(8, 4), title='test_accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(benchmark_df['model_name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(benchmark_df, x='num_model_params', y='model_name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(benchmark_df['model_name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(benchmark_df, x='validation_accuracy', y='model_name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(benchmark_df['model_name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(benchmark_df, x='test_accuracy', y='model_name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)